In [1]:
import ast
import datetime
import os
import random
import re
import warnings

import autogluon.tabular
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection

import dynmodel

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

warnings.filterwarnings("ignore", message="Can't initialize NVML")

In [2]:
DIRECTORY_DATA = 'data/model-autogluon'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

# Preparing data

In [3]:
# FILENAME_DF_ALL_CLEAN = None
#FILENAME_DF_ALL_CLEAN = 'data/20241230_173555/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250128_094430/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250220_094622_halfway/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250220_094622/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250223_150717_halfway/df_all_clean.csv'
FILENAME_DF_ALL_CLEAN = 'data/20250223_150717/df_all_clean.csv'

# FILENAME_DF_ALL_CLEAN_VEHICLES = None
FILENAME_DF_ALL_CLEAN_VEHICLES = 'data/20250223_150717/df_all_clean_vehicles.csv'

In [68]:
SEPARATOR_COL = ': '


def col2parts(col: str) -> tuple[str, str]:
    parts = col.split(SEPARATOR_COL)
    if len(parts) == 1:
        return '', parts[0]
    assert len(parts) == 2
    return parts[0], parts[1]


def load_df_all():
    if FILENAME_DF_ALL_CLEAN_VEHICLES is not None:
        print(FILENAME_DF_ALL_CLEAN_VEHICLES)
        df_all = pd.read_csv(FILENAME_DF_ALL_CLEAN_VEHICLES, header=list(range(2)), index_col=[0, 1])
        df_all.index.set_names(['Scenario ID', 'Vehicle'], inplace=True)
        df_all.columns = [SEPARATOR_COL.join(col) for col in df_all.columns]
        
        # df_all = df_all[df_all.index.get_level_values('Scenario ID').str.contains('slowness no, forcing no')]  # baseline
        df_all = df_all[df_all.index.get_level_values('Vehicle') != 'V0']  # without human
    
    elif FILENAME_DF_ALL_CLEAN is not None:
        print(FILENAME_DF_ALL_CLEAN)
        df_all = pd.read_csv(FILENAME_DF_ALL_CLEAN, header=list(range(2)), index_col=0)
        df_all.index.set_names(['Scenario ID'], inplace=True)
        df_all.columns = [SEPARATOR_COL.join(col) for col in df_all.columns]
        
        df_all = df_all[df_all.index.get_level_values('Scenario ID').str.contains('slowness no, forcing no')]  # baseline
        
    else:
        runnames = [
            '20241203_170129_all600',
            '20241213_104400_racing',
            '20241214_122216_racing_passhum',
        ]
        df_all = pd.concat(
            [pd.read_csv(f'data/{runname}/df_all.csv') 
             for runname in runnames],
            keys=runnames,
            names=['runname', 'row']
        )
        assert all(SEPARATOR_COL not in col for col in df_all.columns), df_all.columns
            
    return df_all

df_all = load_df_all()
df_all.info()

data/20250223_150717/df_all_clean_vehicles.csv
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8658 entries, ('map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities', 'V1') to ('map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50% with forcing 50%', 'V3')
Columns: 523 entries, Violation type: Priority violation to Output of simulation (execution): Near-miss rate
dtypes: bool(4), float64(517), object(2)
memory usage: 34.4+ MB


In [69]:
list(df_all.columns)

['Violation type: Priority violation',
 'Violation type: Speed violation',
 'Violation type: Priority violation and Speed violation',
 'Coordination strategy: Probability of forcing',
 'Coordination strategy: Probability of stops on forcing',
 'Coordination strategy: Rerouting',
 'Static map features: ~ Map ID',
 'Static map features: No. of OPs',
 'Static map features: Connectivity',
 'Static map features: ~ Position',
 'Output of simulation (planning): POD score',
 'Output of simulation (planning): Mean POD score for AVs (V1-V3)',
 'Output of simulation (planning): POD C 0',
 'Output of simulation (planning): POD C 1',
 'Output of simulation (planning): POD C 2',
 'Output of simulation (planning): POD C 3',
 'Output of simulation (planning): POD C 4',
 'Output of simulation (planning): POD C 5',
 'Output of simulation (planning): POD C 6',
 'Output of simulation (planning): POD C 7',
 'Output of simulation (planning): POD C 8',
 'Output of simulation (planning): POD C 9',
 'Output of

In [70]:
ID_EXPERIMENT_TEXT = '110'

In [71]:
ID_EXPERIMENT_TEXT = f'{ID_EXPERIMENT_TEXT:03}'
ID_EXPERIMENT_TEXT_FIRST = '100'
ID_EXPERIMENT = tuple(int(c) for c in ID_EXPERIMENT_TEXT) 


def series2values(series):
    dtype = series.dtype
    if dtype == 'bool':
        return series.astype('int').values
    if dtype in ('int64', 'float64'):
        return series.values
    if dtype == 'object':
        return series.astype('category').cat.codes
    raise TypeError(f'{dtype} is not supported')
    

if FILENAME_DF_ALL_CLEAN_VEHICLES is not None or FILENAME_DF_ALL_CLEAN is not None:
    COLUMNS_NONSPLIT = {
        col: series2values
        for col in [
            'Static map features: ~ Position',  # includes i_map, i_position
        ]
    }
    COLUMNS_INPUT = {col: series2values 
                     # for col in (
                     #    'Static map features: Connectivity',
                     # )}
                     for col in df_all.columns 
                     if col2parts(col)[0] != 'Output of simulation (execution)'
                     and not col2parts(col)[1].startswith('~ ')}
    # COLUMNS_INPUT.pop('Static map features: No. of OPs')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for MV (V0)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V1)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V2)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V3)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Total Mission length for AVs (V1-V3)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Total Mission length for AVs (V1-V3)')  # TODO
    # COLUMNS_INPUT = {key: value for key, value in COLUMNS_INPUT.items()
    #                  if 'No. of OPs' not in key and 'POD' not in key}   
    # COLUMNS_INPUT = {key: value for key, value in COLUMNS_INPUT.items()
    #                  if 'POD C' not in key}      
    COLUMNS_INPUT = {
        col: value for col, value in COLUMNS_INPUT.items()
        if ID_EXPERIMENT[0] and re.search(r': POD score$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r'POD score for AV', col)
        # or ID_EXPERIMENT[1] and re.search(r': POD C \d+$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r': POD C for AV', col)
        or ID_EXPERIMENT[1] and re.search(r': POD D\d \d+$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r': POD D\d for AV', col)
        or ID_EXPERIMENT[2] and re.search(r': Mission length$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r'Mission length for AV', col)
    }
            
    COLUMNS_OUTPUT = {
        col: series2values
        for col in [
            (
                'Output of simulation (execution): No. of completed missions'
                if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else
                'Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)'
            ),
            (
                'Output of simulation (execution): No. of collisions'
                if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else
                'Output of simulation (execution): Collision rate'
            ),
        ]
    }

else:
    COLUMNS_NONSPLIT = {
        'Scenario ID': lambda series: series.values,
    }
    COLUMNS_INPUT = {
        'Coordination strategy': lambda series: series.astype('category').cat.codes.values,
        'isCanPassFirstHum': lambda series: series.astype('int').values,
        'isRacingThroughCrossroadAllowed': lambda series: series.astype('int').values,
        
        'Vehicle ID': lambda series: series.values,
        'Linearization C': lambda series: [
            series.apply(lambda x: -1 if x is None else x[i])
            for i in range(len(series.dropna().iloc[0]))
        ],   
    }
    COLUMNS_OUTPUT = {
        'traveled total, m': lambda series: series.values,
        'No. of completed missions': lambda series: series.values,
        'No. of collisions': lambda series: series.values,
        'No. of near-misses': lambda series: series.values,
    }

COLUMNS_ALL = {**COLUMNS_NONSPLIT, **COLUMNS_INPUT, **COLUMNS_OUTPUT}
COLUMNS_ALL

{'Static map features: ~ Position': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 0': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 1': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 2': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 3': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 4': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 5': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 6': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 7': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD D0 8': <function __main__.series2values(series)>,
 'Output of simulation (planni

In [72]:
df_inout = df_all[list(COLUMNS_ALL)]
df_inout

Static map features: ~ Position  \
Scenario ID                                                                                                                                Vehicle                                   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                         V1                                  1-1   
                                                                                                                                           V2                                  1-1   
                                                                                                                                           V3                                  1-1   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                               V1                                  1-1   
                                                                                                                                           V2                                  1-1   
...                                                                                                                                                                            ...   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50%                  V2                                  9-9   
                                                                                                                                           V3                                  9-9   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50% with forcing 50% V1                                  9-9   
                                                                                                                                           V2                                  9-9   
                                                                                                                                           V3                                  9-9   

                                                                                                                                                    Output of simulation (planning): POD score  \
Scenario ID                                                                                                                                Vehicle                                               
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                         V1                                            0.104   
                                                                                                                                           V2                                            0.058   
                                                                                                                                           V3                                            0.149   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                               V1                                            0.104   
                                                                                                                                           V2                                            0.058   
...                                                                                                                                                                                        ...   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50%                  V2                                            0.348   
                                  

In [73]:
def parse_tuple_string(tuple_string):
    if pd.isna(tuple_string):
        return None
    return ast.literal_eval(tuple_string)


def preprocess_inout(df_inout):
    df_inout = df_inout.copy()
    
    linearization_columns = [col for col in df_inout.columns if isinstance(col, str) and col.startswith('Linearization')]
    for col in linearization_columns:
        df_inout[col] = df_inout[col].apply(parse_tuple_string)
        
    dict_preprocessed = {}
    for col, series2data in COLUMNS_ALL.items():
        data = series2data(df_inout[col])
        if not isinstance(data, list):
            dict_preprocessed[col] = data
        else:
            for i, series in enumerate(data):
                dict_preprocessed[f'{col}#{i}'] = series
    
    return pd.DataFrame(dict_preprocessed)
    

df_preprocessed = preprocess_inout(df_inout)
df_preprocessed

Static map features: ~ Position  \
Scenario ID                                                                                                                                Vehicle                                    
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                         V1                                     0   
                                                                                                                                           V2                                     0   
                                                                                                                                           V3                                     0   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                               V1                                     0   
                                                                                                                                           V2                                     0   
...                                                                                                                                                                             ...   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50%                  V2                                    99   
                                                                                                                                           V3                                    99   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50% with forcing 50% V1                                    99   
                                                                                                                                           V2                                    99   
                                                                                                                                           V3                                    99   

                                                                                                                                                    Output of simulation (planning): POD score  \
Scenario ID                                                                                                                                Vehicle                                               
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                         V1                                            0.104   
                                                                                                                                           V2                                            0.058   
                                                                                                                                           V3                                            0.149   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                               V1                                            0.104   
                                                                                                                                           V2                                            0.058   
...                                                                                                                                                                                        ...   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50%                  V2                                            0.348   
                      

In [74]:
def show(obj, title=None):
    if title is not None:
        display(HTML(f"<h3>{title}</h3>"))
    display(obj)


def shuffle_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    # Shuffle by Scenario ID
    unique_scenarios = df['Scenario ID'].unique()  # Get unique Scenario IDs
    shuffled_scenarios = pd.Series(unique_scenarios).sample(frac=1, random_state=1).tolist()  # Shuffle Scenario IDs
    
    # Reorder the dataframe by the shuffled Scenario IDs
    df_shuffled = pd.concat([df[df['Scenario ID'] == scenario] for scenario in shuffled_scenarios])
    
    # Reset index (optional)
    df_shuffled = df_shuffled.reset_index(drop=True)
    
    return df_shuffled
    """
    return df.sample(frac=1, random_state=1)


def split_train_test(df):
    test_size = 0.2
    
    if not COLUMNS_NONSPLIT:
        return sklearn.model_selection.train_test_split(df, test_size=test_size, random_state=1)
    
    gss = sklearn.model_selection.GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=1)
    column, = list(COLUMNS_NONSPLIT)
    groups = df[column]
    
    # Split the data
    for train_idx, test_idx in gss.split(df, groups=groups):
        return shuffle_df(df.iloc[train_idx]), shuffle_df(df.iloc[test_idx])


df_train, df_test = split_train_test(df_preprocessed)
show(df_train, 'df_train')
show(df_test, 'df_test')
if 'Static map features: ~ Position' in df_train.columns:
    assert not set(df_train['Static map features: ~ Position']) & set(df_test['Static map features: ~ Position']) 

,,Static map features: ~ Position,Output of simulation (planning): POD score,Output of simulation (planning): POD D0 0,Output of simulation (planning): POD D0 1,Output of simulation (planning): POD D0 2,Output of simulation (planning): POD D0 3,Output of simulation (planning): POD D0 4,Output of simulation (planning): POD D0 5,Output of simulation (planning): POD D0 6,Output of simulation (planning): POD D0 7,Output of simulation (planning): POD D0 8,Output of simulation (planning): POD D0 9,Output of simulation (planning): POD D0 10,Output of simulation (planning): POD D0 11,Output of simulation (planning): POD D0 12,Output of simulation (planning): POD D0 13,Output of simulation (planning): POD D0 14,Output of simulation (planning): POD D0 15,Output of simulation (planning): POD D0 16,Output of simulation (planning): POD D0 17,Output of simulation (planning): POD D0 18,Output of simulation (planning): POD D0 19,Output of simulation (planning): POD D0 20,Output of simulation (planning): POD D0 21,Output of simulation (planning): POD D0 22,Output of simulation (planning): POD D0 23,Output of simulation (planning): POD D0 24,Output of simulation (planning): POD D0 25,Output of simulation (planning): POD D0 26,Output of simulation (planning): POD D0 27,Output of simulation (planning): POD D0 28,Output of simulation (planning): POD D0 29,Output of simulation (planning): POD D0 30,Output of simulation (planning): POD D0 31,Output of simulation (planning): POD D0 32,Output of simulation (planning): POD D0 33,Output of simulation (planning): POD D0 34,Output of simulation (planning): POD D0 35,Output of simulation (planning): POD D0 36,Output of simulation (planning): POD D0 37,Output of simulation (planning): POD D0 38,Output of simulation (planning): POD D0 39,Output of simulation (planning): POD D0 40,Output of simulation (planning): POD D0 41,Output of simulation (planning): POD D0 42,Output of simulation (planning): POD D0 43,Output of simulation (planning): POD D0 44,Output of simulation (planning): POD D0 45,Output of simulation (planning): POD D0 46,Output of simulation (planning): POD D0 47,...,Output of simulation (planning): POD D3 52,Output of simulation (planning): POD D3 53,Output of simulation (planning): POD D3 54,Output of simulation (planning): POD D3 55,Output of simulation (planning): POD D3 56,Output of simulation (planning): POD D3 57,Output of simulation (planning): POD D3 58,Output of simulation (planning): POD D3 59,Output of simulation (planning): POD D3 60,Output of simulation (planning): POD D3 61,Output of simulation (planning): POD D3 62,Output of simulation (planning): POD D3 63,Output of simulation (planning): POD D3 64,Output of simulation (planning): POD D3 65,Output of simulation (planning): POD D3 66,Output of simulation (planning): POD D3 67,Output of simulation (planning): POD D3 68,Output of simulation (planning): POD D3 69,Output of simulation (planning): POD D3 70,Output of simulation (planning): POD D3 71,Output of simulation (planning): POD D3 72,Output of simulation (planning): POD D3 73,Output of simulation (planning): POD D3 74,Output of simulation (planning): POD D3 75,Output of simulation (planning): POD D3 76,Output of simulation (planning): POD D3 77,Output of simulation (planning): POD D3 78,Output of simulation (planning): POD D3 79,Output of simulation (planning): POD D3 80,Output of simulation (planning): POD D3 81,Output of simulation (planning): POD D3 82,Output of simulation (planning): POD D3 83,Output of simulation (planning): POD D3 84,Output of simulation (planning): POD D3 85,Output of simulation (planning): POD D3 86,Output of simulation (planning): POD D3 87,Output of simulation (planning): POD D3 88,Output of simulation (planning): POD D3 89,Output of simulation (planning): POD D3 90,Output of simulation (planning): POD D3 91,Output of simulation (planning): POD D3 92,Output of simulation (planning): POD D3 93,Output of simulation (planning): POD D3 94,Output of simula

,,Static map features: ~ Position,Output of simulation (planning): POD score,Output of simulation (planning): POD D0 0,Output of simulation (planning): POD D0 1,Output of simulation (planning): POD D0 2,Output of simulation (planning): POD D0 3,Output of simulation (planning): POD D0 4,Output of simulation (planning): POD D0 5,Output of simulation (planning): POD D0 6,Output of simulation (planning): POD D0 7,Output of simulation (planning): POD D0 8,Output of simulation (planning): POD D0 9,Output of simulation (planning): POD D0 10,Output of simulation (planning): POD D0 11,Output of simulation (planning): POD D0 12,Output of simulation (planning): POD D0 13,Output of simulation (planning): POD D0 14,Output of simulation (planning): POD D0 15,Output of simulation (planning): POD D0 16,Output of simulation (planning): POD D0 17,Output of simulation (planning): POD D0 18,Output of simulation (planning): POD D0 19,Output of simulation (planning): POD D0 20,Output of simulation (planning): POD D0 21,Output of simulation (planning): POD D0 22,Output of simulation (planning): POD D0 23,Output of simulation (planning): POD D0 24,Output of simulation (planning): POD D0 25,Output of simulation (planning): POD D0 26,Output of simulation (planning): POD D0 27,Output of simulation (planning): POD D0 28,Output of simulation (planning): POD D0 29,Output of simulation (planning): POD D0 30,Output of simulation (planning): POD D0 31,Output of simulation (planning): POD D0 32,Output of simulation (planning): POD D0 33,Output of simulation (planning): POD D0 34,Output of simulation (planning): POD D0 35,Output of simulation (planning): POD D0 36,Output of simulation (planning): POD D0 37,Output of simulation (planning): POD D0 38,Output of simulation (planning): POD D0 39,Output of simulation (planning): POD D0 40,Output of simulation (planning): POD D0 41,Output of simulation (planning): POD D0 42,Output of simulation (planning): POD D0 43,Output of simulation (planning): POD D0 44,Output of simulation (planning): POD D0 45,Output of simulation (planning): POD D0 46,Output of simulation (planning): POD D0 47,...,Output of simulation (planning): POD D3 52,Output of simulation (planning): POD D3 53,Output of simulation (planning): POD D3 54,Output of simulation (planning): POD D3 55,Output of simulation (planning): POD D3 56,Output of simulation (planning): POD D3 57,Output of simulation (planning): POD D3 58,Output of simulation (planning): POD D3 59,Output of simulation (planning): POD D3 60,Output of simulation (planning): POD D3 61,Output of simulation (planning): POD D3 62,Output of simulation (planning): POD D3 63,Output of simulation (planning): POD D3 64,Output of simulation (planning): POD D3 65,Output of simulation (planning): POD D3 66,Output of simulation (planning): POD D3 67,Output of simulation (planning): POD D3 68,Output of simulation (planning): POD D3 69,Output of simulation (planning): POD D3 70,Output of simulation (planning): POD D3 71,Output of simulation (planning): POD D3 72,Output of simulation (planning): POD D3 73,Output of simulation (planning): POD D3 74,Output of simulation (planning): POD D3 75,Output of simulation (planning): POD D3 76,Output of simulation (planning): POD D3 77,Output of simulation (planning): POD D3 78,Output of simulation (planning): POD D3 79,Output of simulation (planning): POD D3 80,Output of simulation (planning): POD D3 81,Output of simulation (planning): POD D3 82,Output of simulation (planning): POD D3 83,Output of simulation (planning): POD D3 84,Output of simulation (planning): POD D3 85,Output of simulation (planning): POD D3 86,Output of simulation (planning): POD D3 87,Output of simulation (planning): POD D3 88,Output of simulation (planning): POD D3 89,Output of simulation (planning): POD D3 90,Output of simulation (planning): POD D3 91,Output of simulation (planning): POD D3 92,Output of simulation (planning): POD D3 93,Output of simulation (planning): POD D3 94,Output of simula

In [75]:
df_train.sort_values(by='Scenario ID')

Static map features: ~ Position  \
Scenario ID                                                                                                                                Vehicle                                    
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                         V1                                     0   
                                                                                                                                           V2                                     0   
                                                                                                                                           V3                                     0   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                               V2                                     0   
                                                                                                                                           V3                                     0   
...                                                                                                                                                                             ...   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50%                  V1                                    99   
                                                                                                                                           V3                                    99   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50% with forcing 50% V2                                    99   
                                                                                                                                           V3                                    99   
                                                                                                                                           V1                                    99   

                                                                                                                                                    Output of simulation (planning): POD score  \
Scenario ID                                                                                                                                Vehicle                                               
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                         V1                                            0.104   
                                                                                                                                           V2                                            0.058   
                                                                                                                                           V3                                            0.149   
map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                               V2                                            0.058   
                                                                                                                                           V3                                            0.149   
...                                                                                                                                                                                        ...   
map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50%                  V1                                            0.652   
                      

# Logistic regression (as a baseline)

In [76]:
def split_df_to_X_y(df):
    columns_input_df = [col for col in df.columns
                        if (col if not isinstance(col, str) else col.split('#')[0]) in COLUMNS_INPUT]
    columns_output_df = list(COLUMNS_OUTPUT)
    assert set(COLUMNS_NONSPLIT) | set(columns_input_df) | set(columns_output_df) == set(df.columns)
    
    X = df[columns_input_df]
    y = df[columns_output_df]    
    return X, y


def run_regression(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    ndarray_predictions = model.predict(X_test)
    df_predictions = pd.DataFrame(ndarray_predictions, columns=y_test.columns)
    return df_predictions


df_predictions_regression = run_regression(df_train, df_test)
df_predictions_regression

,Output of simulation (execution): No. of completed missions,Output of simulation (execution): No. of collisions
0,3.787033,2.446461
1,7.956564,0.264010
2,133.198592,58.413471
3,8.248557,0.293859
4,5.459713,4.366358
...,...,...
1726,4.408714,1.714870
1727,5.393919,1.626426
1728,7.379202,0.133014
1729,9.458751,2.410036


In [77]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}-{random.random()}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    display(HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" title="{datetime.datetime.now()}" />'))
    
    plt.close(fig)
    
    return filename


def evaluate_and_plot_column(df_test, df_predictions, column, *, is_plot=True):
    y_test_column = df_test[column]
    predictions_column = df_predictions[column]
    
    r2 = sklearn.metrics.r2_score(y_test_column, predictions_column)
    name = col2parts(column)[1]
    print(f"{name}:")
    print(f"- R^2 Score: {r2}")
    
    if not is_plot:
        return

    # Plot results for each output column
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(y_test_column, predictions_column, color='blue', alpha=0.5)
    plt.plot([y_test_column.min(), y_test_column.max()], [y_test_column.min(), y_test_column.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {name}')
    plt.grid(True)
    save_and_show(fig, f'Actual_vs_Predicted_Values_{name}')


def evaluate_and_plot_all_columns(df_test, df_predictions):
    for column in COLUMNS_OUTPUT:
        evaluate_and_plot_column(df_test, df_predictions, column, is_plot=False)
        
        
evaluate_and_plot_all_columns(df_test, df_predictions_regression)

No. of completed missions:
- R^2 Score: -29.856844164403373
No. of collisions:
- R^2 Score: -9.70476004190088


# AutoGluon

In [78]:
def run_autogluon(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    # Train AutoGluon models
    predictors = []
    df_predictions = pd.DataFrame()
    for i_column, column in enumerate(COLUMNS_OUTPUT):
        print(f'{column=}:')
        df_train_predictor = pd.concat([X_train, y_train[[column]]], axis=1)
        preset = 'medium'  # medium (~1 min.), good (~15 min.), high (~2 h)
        
        predictor = autogluon.tabular.TabularPredictor(
            label=column, 
            eval_metric='r2', 
            problem_type='regression',
        ).fit(
            df_train_predictor,
            presets=preset,
            hyperparameters={
                'GBM': {},       # LightGBM (TODO: something like `GBMLarge`)
                'XGB': {},       # XGBoost
                'RF': {},        # Random Forest
                'XT': {},        # Extra Trees
                # 'CAT': {},      # CatBoost, omitted if slow
                # 'NN': {},       # Neural net, if you want it
                # 'LR': {},       # Linear model
                # 'KNN': {},      # K-Nearest Neighbors
            },
        )
        predictors.append(predictor)
        
        df_predictions[column] = predictor.predict(X_test)

        # Leaderboard - Display a table of different models and their performance
        df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        leaderboard = predictor.leaderboard(df_test_predictor, silent=True)
        rundir = os.path.dirname(FILENAME_DF_ALL_CLEAN)
        _leaderboard, r2 = dynmodel.process_leaderboard(leaderboard, rundir, column, f'{preset} {ID_EXPERIMENT_TEXT}')        
        
        filename_results = f'{rundir}/{preset} results.csv' 
        with open(
                filename_results, 
                'w' if ID_EXPERIMENT_TEXT == ID_EXPERIMENT_TEXT_FIRST and i_column == 0 else 'a'
        ) as file:
            if i_column == 0:
                file.write(ID_EXPERIMENT_TEXT)
            file.write(f',{r2:.3f}')
            if i_column == len(COLUMNS_OUTPUT) - 1:
                file.write('\n')
        
        # Feature importance on training data
        # show(
        #     predictor.feature_importance(df_train_predictor),
        #     'feature_importance(df_train_predictor)'
        # )
        # 
        # # Feature importance on test data
        # show(
        #     predictor.feature_importance(df_test_predictor),
        #     'feature_importance(df_test_predictor)'
        # )
        # Example: SHAP values for a specific model
        # shap_values = predictor.get_model_shap_values(df_test_predictor, model='LightGBM')
        # show(shap_values, 'shap_values')  # SHAP values for each feature and each prediction
        
    return predictors, df_predictions


predictors, df_predictions_autogluon = run_autogluon(df_train, df_test)
df_predictions_autogluon

No path specified. Models will be saved in: "AutogluonModels/ag-20250227_104632"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #141~20.04.1-Ubuntu SMP Thu Jan 16 18:38:51 UTC 2025
CPU Count:          16
Memory Avail:       4.50 GB / 31.09 GB (14.5%)
Disk Space Avail:   204.86 GB / 693.60 GB (29.5%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250227_104632"
Train Data Rows:    6927
Train Data Columns: 401
Label Column:       Output of simulation (execution): No. of completed missions
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:        

column='Output of simulation (execution): No. of completed missions':


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 7 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 8): ['Output of simulation (planning): POD D0 49', 'Output of simulation (planning): POD D0 50', 'Output of simulation (planning): POD D2 0', 'Output of simulation (planning): POD D2 99', 'Output of simulation (planning): POD D3 0', 'Output of simulation (planning): POD D3 1', 'Output of simulation (planning): POD D3 98', 'Output of simulation (planning): POD D3 99']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
3,LightGBM,0.627,0.858,r2,0.006,0.004,4.902,0.006,0.004,4.902,1,True,1
4,WeightedEnsemble_L2,0.627,0.858,r2,0.009,0.004,4.965,0.003,0.001,0.062,2,True,5
2,XGBoost,0.635,0.857,r2,0.021,0.007,6.497,0.021,0.007,6.497,1,True,4
1,RandomForest,0.639,0.856,r2,0.072,0.070,5.817,0.072,0.070,5.817,1,True,2
0,ExtraTrees,0.644,0.856,r2,0.075,0.081,3.962,0.075,0.081,3.962,1,True,3


No path specified. Models will be saved in: "AutogluonModels/ag-20250227_104715"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #141~20.04.1-Ubuntu SMP Thu Jan 16 18:38:51 UTC 2025
CPU Count:          16
Memory Avail:       3.92 GB / 31.09 GB (12.6%)
Disk Space Avail:   204.79 GB / 693.60 GB (29.5%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250227_104715"
Train Data Rows:    6927
Train Data Columns: 401
Label Column:       Output of simulation (execution): No. of collisions
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                

column='Output of simulation (execution): No. of collisions':


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 7 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 8): ['Output of simulation (planning): POD D0 49', 'Output of simulation (planning): POD D0 50', 'Output of simulation (planning): POD D2 0', 'Output of simulation (planning): POD D2 99', 'Output of simulation (planning): POD D3 0', 'Output of simulation (planning): POD D3 1', 'Output of simulation (planning): POD D3 98', 'Output of simulation (planning): POD D3 99']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
1,WeightedEnsemble_L2,0.168,0.140,r2,0.034,0.009,6.738,0.005,0.001,0.068,2,True,5
0,LightGBM,0.168,0.140,r2,0.005,0.002,2.888,0.005,0.002,2.888,1,True,1
4,XGBoost,0.162,0.138,r2,0.024,0.006,3.781,0.024,0.006,3.781,1,True,4
2,RandomForest,0.167,0.086,r2,0.091,0.070,5.840,0.091,0.070,5.840,1,True,2
3,ExtraTrees,0.166,0.086,r2,0.102,0.082,3.397,0.102,0.082,3.397,1,True,3


,,Output of simulation (execution): No. of completed missions,Output of simulation (execution): No. of collisions
Scenario ID,Vehicle,,
"map-generator/generated-maps/3_with_bridges/scenario8-1.json, passhum 0, slowness no, forcing stops 50% with forcing 50%",V1,4.730600,2.399547
"map-generator/generated-maps/3_with_bridges/scenario9-3.json, passhum 0, slowness without rerouting, forcing stops 50% with forcing 50%",V2,7.262696,0.491470
"map-generator/generated-maps/3_without_bridges/scenario9-2.json, passhum 0, slowness without rerouting, forcing no",V1,2.738513,2.573056
"map-generator/generated-maps/3_without_bridges/scenario9-3.json, passhum 0, slowness without rerouting, forcing ignoring human",V2,7.313369,0.491470
"map-generator/generated-maps/3_without_bridges/scenario9-3.json, passhum 0, slowness no, forcing change of priorities",V1,2.737807,2.474530
...,...,...,...
"map-generator/generated-maps/3_with_bridges/scenario8-1.json, passhum 0, slowness no, forcing no",V2,4.402890,1.065454
"map-generator/generated-maps/3_with_bridges/scenario8-4.json, passhum 0, slowness without rerouting, forcing stops",V3,4.333346,0.942970
"map-generator/generated-maps/3_without_bridges/scenario3-10.json, passhum 0, slowness no, forcing stops",V2,6.818292,0.575067


## evaluate_and_plot_all_columns

In [79]:
import collections

def print_stats():
    for col in [
        'Output of simulation (execution): No. of collisions',
    ]:
        print(col)
        for df in [
            df_train,
            df_test,
            df_predictions_autogluon,
        ]:
            print(collections.Counter(df[col]))
        
        
print_stats()

Output of simulation (execution): No. of collisions
Counter({0.0: 3967, 1.0: 1069, 2.0: 578, 3.0: 343, 4.0: 290, 5.0: 210, 6.0: 143, 7.0: 79, 8.0: 72, 10.0: 62, 9.0: 58, 11.0: 32, 12.0: 13, 13.0: 7, 14.0: 3, 15.0: 1})
Counter({0.0: 982, 1.0: 272, 2.0: 155, 3.0: 81, 4.0: 63, 5.0: 50, 6.0: 32, 10.0: 23, 8.0: 22, 7.0: 21, 9.0: 18, 11.0: 7, 12.0: 3, 13.0: 1, 14.0: 1})
Counter({0.7335631847381592: 92, 0.5480876564979553: 45, 0.7013294100761414: 41, 0.5507568120956421: 34, 0.7350867390632629: 33, 1.0331465005874634: 33, 0.4914698600769043: 29, 0.4519071578979492: 29, 0.8332486152648926: 29, 0.5206869840621948: 29, 0.5750667452812195: 24, 2.3995466232299805: 17, 1.4689418077468872: 17, 1.4883402585983276: 17, 2.4068636894226074: 17, 0.5490272045135498: 17, 2.4857001304626465: 17, 1.4145138263702393: 17, 1.065454363822937: 17, 0.7165493369102478: 17, 1.0440272092819214: 17, 1.3860101699829102: 17, 1.4205400943756104: 17, 0.9659252166748047: 17, 0.927096962928772: 17, 0.7107933759689331: 17, 1.

In [80]:
evaluate_and_plot_all_columns(df_test, df_predictions_autogluon)

No. of completed missions:
- R^2 Score: 0.6266914172914072
No. of collisions:
- R^2 Score: 0.16802633541261724


## explain_predictions

In [81]:
def explain_predictions(predictors):
    # X_test, y_test = split_df_to_X_y(df_test)
    
    for column, predictor in zip(COLUMNS_OUTPUT, predictors):
        # df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        if column not in (
            'Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)',
            'Output of simulation (execution): Collision rate',
        ):
            continue
        
        for model in 'LightGBM', 'XGBoost':
            tree_model = predictor._trainer.load_model(model)
            if model == 'LightGBM':
                tree_importance = tree_model.model.feature_importance(importance_type='gain')
            elif model == 'XGBoost':
                tree_importance = tree_model.model.feature_importances_
            else:
                raise ValueError(model)
            # show(tree_importance, column)
                
            
            feature_names = predictor.feature_metadata.get_features()
            df = pd.DataFrame.from_dict(
                {name: {'importance': value} for name, value in zip(feature_names, tree_importance)},
                orient='index'
            )
            # print(df.index)
            # Group indexes by their base name before `#`
            df['group'] = df.index.str.extract(r'^(.+?)(?:#\d+)?$', expand=False)
            df = df.groupby('group')['importance'].sum().to_frame()
            
            df.sort_values(by='importance', ascending=False, inplace=True)
            show(df, f'{model}: {column}')
        
        
explain_predictions(predictors)